# Naive Bayes

Материалы: <br>
Naive Bayes Classification and sentiment (https://web.stanford.edu/~jurafsky/slp3/4.pdf)

## Корпус + предобработка

В качестве данных возьмем мини-корпус твитов о банках. За его основу был взят корпус отсюда: http://www.dialog-21.ru/evaluation/2016/sentiment/ 

In [ ]:
import pandas as pd
from nltk.tokenize import TweetTokenizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from pymystem3 import Mystem
m = Mystem()

Installing mystem to /root/.local/bin/mystem from http://download.cdn.yandex.net/mystem/mystem-3.1-linux-64bit.tar.gz


In [ ]:
df = pd.read_csv('bank_sent.csv', sep=';')

In [ ]:
df = df.drop(columns='Unnamed: 0')

In [ ]:
df

,text,label
0,http://t.co/YEVHuvVGA1 Взять кредит тюмень аль...,0
1,Мнение о кредитной карте втб 24 http://t.co/SB...,0
2,«Райффайзенбанк»: Снижение ключевой ставки ЦБ ...,0
3,Современное состояние кредитного поведения в р...,0
4,http://t.co/Qr6JbSVTxY Оформить краткосрочный ...,0
...,...,...
8530,#newz_day Банк УРАЛСИБ повысил ставки по вклад...,1
8531,"хэй.я знаю,что ты заказывала одну вещь с офф с...",0
8532,Это радует - Банк Уралсиб и Фольксваген банк Р...,0
8533,При финансовой поддержке Россельхозбанка постр...,0


Для упрощения задачи выберем из данных только положительные и отрицательные отзывы.

In [ ]:
df = df[df.label != 0]

In [ ]:
df

,text,label
5,Самый выгодный автокредит в втб 24 http://t.co...,1
8,http://t.co/h6r6GdBe4H Легко можно получить де...,1
12,@ShtirliZ_ @Zhukova_olga @winzard @tereshenkov...,-1
23,"Канада ввела санкции против Газпромбанка, ВЭБа...",-1
37,Снижение процентной ставки по кредиту на недви...,1
...,...,...
8526,#nsk-online Банк УРАЛСИБ повысил ставки по вкл...,1
8527,#нсо-потал Банк УРАЛСИБ повысил ставки по вкла...,1
8528,#сводка Банк УРАЛСИБ повысил ставки по вкладам...,1
8529,#добавить_новость Банк УРАЛСИБ повысил ставки ...,1


Разделим датасет на обучающую и тестовую выборки.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df['text'], df['label'], test_size=0.33, random_state = 42)

In [ ]:
X_train

5379    RT @rodoss: Сбербанк, сними рекламу, где патла...
6002    Копии паспортов клиентов Сбербанка выбросили н...
8089    @sberbank просто душечка - мобильный банк восс...
7382                         @alfa_bank Спасибо за ответ.
2531    RT @stikskure: ВТБ 24 прекратил прием консульс...
                              ...                        
5791    ЦБ РФ продолжает бороться с банками,проводящим...
6366    @sberbank вы же пониамете что впши 6.5% годовы...
4861    Вот есть у меня на карте деньги, а снять я их ...
7551    вот такие очереди к банкоматам от @alfa_bank с...
5784    так зачем отстаивать очереди и платить гигантс...
Name: text, Length: 1153, dtype: object

Приведем к нижнему регистру и токенизируем

In [ ]:
tokenizer = TweetTokenizer()
X_train = X_train.str.lower()
X_train = X_train.apply(tokenizer.tokenize)
X_test = X_test.str.lower()
X_test = X_test.apply(tokenizer.tokenize)

In [ ]:
 !pip install pymorphy2

     |████████████████████████████████| 61kB 4.6MB/s 
     |████████████████████████████████| 8.2MB 8.0MB/s 


In [ ]:
import nltk
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

## Naive Bayes algorithm

In [ ]:
import math
import numpy as np
from collections import Counter
from nltk.corpus import stopwords
import pymorphy2

def preprocessing(array_of_words, Lemmatization, delete_stop_w):
  ans = array_of_words
  if(Lemmatization == True):
    ans = lemmatization(array_of_words)
  
  if(delete_stop_w == True):
    ans = delete_stop_words(array_of_words)

  return ans


def lemmatization(array_of_words):
    ans = []
    morph = pymorphy2.MorphAnalyzer()
    for word in array_of_words:
        ans.append(morph.parse(word)[0].normal_form)
    return ans

def delete_stop_words(array_of_words):
    filtered_tokens = []
    stop_words = stopwords.words("russian")
    for i in ['хорошо', 'никогда', 'лучше', 'нельзя']:
      stop_words.remove(i)
    for token in array_of_words:
      if token not in stop_words:
        filtered_tokens.append(token)
    return filtered_tokens



class NaiveBayes(object):
    
    def __init__(self, p_c, p_w_c):
        """Initialises a new classifier."""
        self.p_c = p_c  #{1: 0.39375542064180397, -1: 0.606244579358196}
        self.p_w_c = p_w_c #{'word': {1: 0.1, -1: 0.2}, 'word_2': {}, ...}

    def predict(self, x, sentiment_lexicon = None, Lemmatization = False, delete_stop_w = False):
        """Predicts the class for a document.
        
        Args:
            x: A document, represented as a list of words.

        Returns:
            The predicted class, represented as an integer (1 or -1).
        """
        # ВАШ КОД (формула 4.10)
        if (sentiment_lexicon == None):

          val_c0 = 0
          val_c1 = 0
          x = preprocessing(x, Lemmatization, delete_stop_w)           #lemmatization
          for word in x:
            if (self.p_w_c[-1][word] != 0 and self.p_w_c[1][word] != 0): # Ignore unknown words
              val_c0 += np.log(self.p_w_c[-1][word])
              val_c1 += np.log(self.p_w_c[1][word])

          if np.log(self.p_c[1]) +  val_c1 > np.log(self.p_c[-1]) +  val_c0:
            best_class = 1
          else:
            best_class = -1 

          return best_class

        else:

          val_c0 = 0
          val_c1 = 0
          x = preprocessing(x, Lemmatization, delete_stop_w)            #lemmatization
          for word in x:
            if (word in sentiment_lexicon.keys()):

              if sentiment_lexicon[word] == 'negative':
                val_c0 += np.log(0.999999999)
                val_c1 += np.log(0.000000001)

              if sentiment_lexicon[word] == 'positive':
                val_c1 += np.log(0.999999999)
                val_c0 += np.log(0.000000001)

            elif (self.p_w_c[-1][word] != 0 and self.p_w_c[1][word] != 0): # Ignore unknown words
                val_c0 += np.log(self.p_w_c[-1][word])
                val_c1 += np.log(self.p_w_c[1][word])

          if np.log(self.p_c[1]) +  val_c1 > np.log(self.p_c[-1]) +  val_c0:
            best_class = 1
          else:
            best_class = -1 

          return best_class

    
        
    @classmethod # принимает на вход класс (а не объект класса)

    def train(cls, x, y, Lemmatization = False, delete_stop_w = False):
        """Train a new classifier on training data.

        Args:
            cls: The Python class representing the classifier.
            x, y: Training data and labels.

        Returns:
            A trained classifier, an instance of `cls`.
        """
        # ВАШ КОД
        
        
        p_c = {1: y_train[y_train == 1].count()/y_train.shape[0], -1: y_train[y_train == -1].count()/y_train.shape[0]} # формула 4.11

        morph = pymorphy2.MorphAnalyzer()

        # generate dict for everyone word in c_1 and c_-1
        words_c1 = []
        for sentense in X_train[y_train == 1]:
            words_c1 += sentense
        
        words_c1 = preprocessing(words_c1, Lemmatization, delete_stop_w)
        dict_words_c1 = Counter(words_c1)
        cnt_all_words_c1 = sum(dict_words_c1.values())


        words_c0 = []
        for sentense in X_train[y_train == -1]:
            words_c0 += sentense

        words_c0 = preprocessing(words_c0, Lemmatization, delete_stop_w)
        dict_words_c0 = Counter(words_c0)
        cnt_all_words_c0 = sum(dict_words_c0.values())

        # add-one smoothing

        V = list(set(words_c0 + words_c1))

        for word in V:
          dict_words_c1[word] = (dict_words_c1[word] + 1) / (cnt_all_words_c1 + len(V))

        for word in V:
          dict_words_c0[word] = (dict_words_c0[word] + 1) / (cnt_all_words_c0 + len(V))

        
            
        p_w_c = {1 : dict_words_c1, -1 : dict_words_c0} # формула 4.12 (в знаменателе -- тексты определенного класса)
        
        # не забудьте применить add-one smoothing (формула 4.14)
        
        return cls(p_c, p_w_c)

In [ ]:
NB_classifier = NaiveBayes.train(X_train, y_train)

In [ ]:
NB_classifier.p_c

{-1: 0.606244579358196, 1: 0.39375542064180397}

In [ ]:
NB_classifier.p_w_c

{-1: Counter({'rt': 0.008811594202898551,
          '@rodoss': 0.00011594202898550724,
          ':': 0.012985507246376812,
          'сбербанк': 0.015072463768115942,
          ',': 0.028405797101449276,
          'сними': 0.00011594202898550724,
          'рекламу': 0.00017391304347826088,
          'где': 0.0002898550724637681,
          'патлатые': 0.00011594202898550724,
          'мужики': 0.00011594202898550724,
          'в': 0.020753623188405797,
          'кожанках': 0.00011594202898550724,
          'рекомендуют': 0.00011594202898550724,
          'открыть': 0.00017391304347826088,
          'металлический': 0.00011594202898550724,
          'счет': 0.0005797101449275362,
          'и': 0.011478260869565217,
          'мечут': 0.00011594202898550724,
          'козы': 0.00011594202898550724,
          'воздух': 0.00011594202898550724,
          'копии': 0.00011594202898550724,
          'паспортов': 0.00017391304347826088,
          'клиентов': 0.0009855072463768116,
       

In [ ]:
y_true = list(y_test)
y_pred = []
for tweet in X_test:
  y_pred.append(NB_classifier.predict(tweet,  sentiment_lexicon, Lemmatization = True, delete_stop_w = True))

In [ ]:
print(classification_report(y_true, y_pred))

              precision    recall  f1-score   support

          -1       0.85      0.91      0.87       369
           1       0.80      0.69      0.74       199

    accuracy                           0.83       568
   macro avg       0.82      0.80      0.81       568
weighted avg       0.83      0.83      0.83       568



## Дополнения модели

Дополните модель различными способами и измерьте качество полученных классификаторов. <br>
Возможные дополнения:
<ul><li> Binary NB (стр 8)
    <li> Добавление отрицания к последующим словам
    <li> (*) Использование словаря с оценочной лексикой (http://www.labinform.ru/pub/rusentilex/rusentilex_2017.txt)</ul>  
Также можно добавить следующие шаги в препроцессинг (любые два):
<ul><li> Лемматизация слов
    <li> Удаление стоп-слов/наиболее частотных слов
    <li> Удаление знаков препинания, служебных символов
</ul> 
Сделайте пару комбинаций, применив несколько улучшений одновременно.

In [ ]:
import urllib.request  # the lib that handles the url stuff
sentiment_lexicon = {}
      
for n, line in  enumerate(urllib.request.urlopen("http://www.labinform.ru/pub/rusentilex/rusentilex_2017.txt"), 1):
    string = (line.decode('utf-8')).split(', ')
    # print(string, n)
    if(n > 18):
      
      
      if (' ' not in string[0]):
        if(string[-2]) == 'negative' or (string[-2]) == 'positive': 
          sentiment_lexicon[string[0]] = string[-2]
        # print(string[0], string[-2])


In [ ]:
ot = 'все просто замечательно'.lower()
ot = tokenizer.tokenize(ot)

In [ ]:
NB_classifier.predict(ot, sentiment_lexicon, Lemmatization = True, delete_stop_w = False)

1

In [ ]:
NB_classifier.predict(ot, sentiment_lexicon, Lemmatization = True, delete_stop_w = True)

-1

In [ ]:
y_true = list(y_test)
y_pred = []
for tweet in X_test:
  y_pred.append(NB_classifier.predict(tweet))

In [ ]:
modifications = {}

In [ ]:
y_true = list(y_test)
y_pred = []
for tweet in X_test:
  y_pred.append(NB_classifier.predict(tweet))
modifications['no_preproc'] = accuracy_score(y_true, y_pred)

y_pred = []
for tweet in X_test:
  y_pred.append(NB_classifier.predict(tweet, sentiment_lexicon, delete_stop_w=True))
modifications['sentiment_lexicon delete_stop_words'] = accuracy_score(y_true, y_pred)

y_pred = []
for tweet in X_test:
  y_pred.append(NB_classifier.predict(tweet, sentiment_lexicon, Lemmatization=True))
modifications['sentiment_lexicon Lemmatization'] = accuracy_score(y_true, y_pred)

y_pred = []
for tweet in X_test:
  y_pred.append(NB_classifier.predict(tweet, delete_stop_w=True, Lemmatization=True))
modifications['delete_stop_words Lemmatization'] = accuracy_score(y_true, y_pred)

y_pred = []
for tweet in X_test:
  y_pred.append(NB_classifier.predict(tweet, sentiment_lexicon, delete_stop_w=True, Lemmatization=True))
modifications['sentiment_lexicon delete_stop_words Lemmatization'] = accuracy_score(y_true, y_pred)


modifications

{'delete_stop_words Lemmatization': 0.8380281690140845,
 'no_preproc': 0.8415492957746479,
 'sentiment_lexicon Lemmatization': 0.7922535211267606,
 'sentiment_lexicon delete_stop_words': 0.8309859154929577,
 'sentiment_lexicon delete_stop_words Lemmatization': 0.8309859154929577}